In [1]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [25]:
# Create two DataFrames to join
df_lhs = session.create_dataframe([["a", 1], ["b", 2],["c", 2]], schema=["key", "value1"])
df_rhs = session.create_dataframe([["a", 3], ["b", 4], ["c", 5]], schema=["key", "value2"])

In [21]:
df_lhs.show()

--------------------
|"KEY"  |"VALUE1"  |
--------------------
|a      |1         |
|b      |2         |
|c      |2         |
--------------------



In [22]:
df_rhs.show()

--------------------
|"KEY"  |"VALUE2"  |
--------------------
|a      |3         |
|b      |4         |
|c      |5         |
--------------------



In [23]:

df_lhs.join(df_rhs, df_lhs.col("key") == df_rhs.col("key"))

In [24]:
df_lhs.show()

--------------------
|"KEY"  |"VALUE1"  |
--------------------
|a      |1         |
|b      |2         |
|c      |2         |
--------------------



In [26]:
df_lhs.join(df_rhs, ["key"]).show()

-------------------------------
|"KEY"  |"VALUE1"  |"VALUE2"  |
-------------------------------
|a      |1         |3         |
|b      |2         |4         |
|c      |2         |5         |
-------------------------------



In [27]:
df_lhs = session.create_dataframe([["a", 1], ["b", 2]], schema=["key", "value1"])
df_rhs = session.create_dataframe([["a", 3], ["b", 4]], schema=["key", "value2"])
# Use & operator connect join expression. '|' and ~ are similar.
df_joined_multi_column = df_lhs.join(df_rhs, (df_lhs.col("key") == df_rhs.col("key")) & (df_lhs.col("value1") < df_rhs.col("value2"))).select(df_lhs["key"].as_("key"), "value1", "value2")
df_joined_multi_column.show()

-------------------------------
|"KEY"  |"VALUE1"  |"VALUE2"  |
-------------------------------
|a      |1         |3         |
|b      |2         |4         |
-------------------------------



In [28]:
df_lhs = session.create_dataframe([["a", 1], ["b", 2]], schema=["key", "value1"])
df_rhs = session.create_dataframe([["a", 3], ["b", 4]], schema=["key", "value2"])
df_lhs.join(df_rhs, df_lhs.col("key") == df_rhs.col("key"), lsuffix="_left", rsuffix="_right").show()

--------------------------------------------------
|"KEY_LEFT"  |"VALUE1"  |"KEY_RIGHT"  |"VALUE2"  |
--------------------------------------------------
|a           |1         |a            |3         |
|b           |2         |b            |4         |
--------------------------------------------------

